<a href="https://colab.research.google.com/github/cshmzin/nlp-code/blob/main/%E6%96%87%E6%9C%AC%E7%9B%B8%E4%BC%BC%E5%BA%A6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import pandas as pd

df_train = pd.read_csv("train_1.csv",delimiter="\t",header=None)
df_test = pd.read_csv("test_1.csv",delimiter="\t",header=None)

print("训练样本数量：", len(df_train))
print("测试样本数量：", len(df_test))

df_train.head()
df_test.head()

训练样本数量： 5000
测试样本数量： 5000


,0,1
0,玩梦幻西游能赚钱吗,梦幻西游2不花钱能玩吗
1,夏天去什么地方旅游好,夏季去什么地方旅游最好（国内的地方）
2,为什么梦幻西游网站打不开,为什么下载梦幻西游游戏补丁网页打不开
3,这对双胞胎像不,像这样的可爱卡通图片要男的谢了
4,免费网络游戏都有哪些?,有哪些免费的网络游戏


In [31]:
from torch.utils.data import Dataset
#!pip install transformers tqdm
from transformers import BertTokenizer
#!pip install pysnooper -q
import pysnooper
import torch

PRETRAINED_MODEL_NAME = "hfl/chinese-roberta-wwm-ext"  
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

class FakeNewsDataset(Dataset):
    def __init__(self, mode, tokenizer,Frist,End):
        assert mode in ["train", "test"]
        self.mode = mode
        self.df = pd.read_csv(f"{self.mode}_1.csv",delimiter="\t",header=None).iloc[Frist:End,:]
        self.tokenizer = tokenizer  #使用 BERT tokenizer

    #@pysnooper.snoop()  # 加入以了解所有转换过程
    def __getitem__(self, idx):
        if self.mode == "test":
            text_a, text_b = self.df.iloc[idx, :2].values
            label_tensor = None
        else:
            text_a, text_b, label = self.df.iloc[idx, :].values
            label_tensor = torch.tensor(label)
            
        word_pieces = ["[CLS]"]
        tokens_a = self.tokenizer.tokenize(text_a)
        word_pieces += tokens_a + ["[SEP]"]
        len_a = len(word_pieces)
        
        tokens_b = self.tokenizer.tokenize(text_b)
        word_pieces += tokens_b + ["[SEP]"]
        len_b = len(word_pieces) - len_a
        
        ids = self.tokenizer.convert_tokens_to_ids(word_pieces)
        tokens_tensor = torch.tensor(ids)
        
        segments_tensor = torch.tensor([0] * len_a + [1] * len_b,dtype=torch.long)
        
        return (tokens_tensor, segments_tensor, label_tensor)
    
    def __len__(self):
        return len(self.df)

    def __iter__(self):
        return iter(self.df)
    
trainset = FakeNewsDataset("train",tokenizer,0,5000)
devset = FakeNewsDataset("train",tokenizer,4500,5000)
testset = FakeNewsDataset("test",tokenizer,0,5000)
print(trainset[1])
print(devset[1])
print(testset[1])
print(len(trainset),len(devset),len(testset))

(tensor([ 101, 2582,  720, 3118,  802, 2140, 5308, 2137, 7213, 6121, 1305, 8043,
         102, 7213, 6121, 1305, 2582,  720, 5308, 2137, 3118,  802, 2140,  102]), tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), tensor(1))
(tensor([ 101,  711,  784,  720, 3766,  782, 4415, 2769,  102,  711,  784,  720,
        3766,  782, 4415, 2769, 1450,  102]), tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]), tensor(1))
(tensor([ 101, 1909, 1921, 1343,  784,  720, 1765, 3175, 3180, 3952, 1962,  102,
        1909, 2108, 1343,  784,  720, 1765, 3175, 3180, 3952, 3297, 1962, 8020,
        1744, 1079, 4638, 1765, 3175, 8021,  102]), tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1]), None)
5000 500 5000


/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


In [32]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

def create_mini_batch(samples):
    tokens_tensors = [s[0] for s in samples]
    segments_tensors = [s[1] for s in samples]
    
    if samples[0][2] is not None:
        label_ids = torch.stack([s[2] for s in samples])
    else:
        label_ids = None
    
    # zero pad 到同一序列长度
    tokens_tensors = pad_sequence(tokens_tensors,batch_first=True)
    segments_tensors = pad_sequence(segments_tensors,batch_first=True)
    
    # attention masks，将 tokens_tensors 不为 zero padding 的位置设为1
    masks_tensors = torch.zeros(tokens_tensors.shape,dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(tokens_tensors != 0, 1)
    
    return tokens_tensors, segments_tensors, masks_tensors, label_ids

BATCH_SIZE = 64
trainloader = DataLoader(trainset,batch_size=BATCH_SIZE,collate_fn=create_mini_batch)
devloader = DataLoader(devset,batch_size=BATCH_SIZE,collate_fn=create_mini_batch)

In [33]:
from transformers import BertForSequenceClassification
from IPython.display import clear_output
PRETRAINED_MODEL_NAME = "hfl/chinese-roberta-wwm-ext"
NUM_LABELS = 2

model = BertForSequenceClassification.from_pretrained(
    PRETRAINED_MODEL_NAME, num_labels=NUM_LABELS)

clear_output()

In [34]:
def get_predictions(model, dataloader, compute_acc=False):
    predictions = None
    correct = 0
    total = 0
      
    with torch.no_grad():
        for data in dataloader:
            if next(model.parameters()).is_cuda:
                data = [t.to("cuda:0") for t in data if t is not None]
            
            tokens_tensors, segments_tensors, masks_tensors = data[:3]
            outputs = model(input_ids=tokens_tensors,token_type_ids=segments_tensors,attention_mask=masks_tensors)
            
            logits = outputs[0]
            _, pred = torch.max(logits.data, 1)
            
            if compute_acc:
                labels = data[3]
                total += labels.size(0)
                correct += (pred == labels).sum().item()
                
            if predictions is None:
                predictions = pred
            else:
                predictions = torch.cat((predictions, pred))
    
    if compute_acc:
        acc = correct / total
        return predictions, acc
    return predictions
    
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)
model = model.to(device)
_, acc = get_predictions(model, devloader, compute_acc=True)
print("classification acc:", acc)

device: cuda:0
classification acc: 0.58


In [35]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
EPOCHS = 10
for epoch in range(EPOCHS):  
    running_loss = 0.0
    model.train()
    for data in trainloader:
        tokens_tensors, segments_tensors, \
        masks_tensors, labels = [t.to(device) for t in data]
        optimizer.zero_grad()
        outputs = model(input_ids=tokens_tensors, 
                        token_type_ids=segments_tensors, 
                        attention_mask=masks_tensors, 
                        labels=labels)

        loss = outputs[0]
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    model.eval()    
    _, train_acc = get_predictions(model, trainloader, compute_acc=True)
    _, dev_acc = get_predictions(model, devloader, compute_acc=True)
    print('[epoch %d] loss: %.3f, train_acc: %.3f, test_acc: %.3f' %
          (epoch + 1, running_loss,train_acc,dev_acc))

[epoch 1] loss: 41.268, train_acc: 0.897, test_acc: 0.924
[epoch 2] loss: 20.328, train_acc: 0.935, test_acc: 0.962
[epoch 3] loss: 14.267, train_acc: 0.964, test_acc: 0.982
[epoch 4] loss: 10.423, train_acc: 0.966, test_acc: 0.990
[epoch 5] loss: 7.688, train_acc: 0.922, test_acc: 0.988
[epoch 6] loss: 7.974, train_acc: 0.902, test_acc: 0.968
[epoch 7] loss: 6.147, train_acc: 0.982, test_acc: 0.992
[epoch 8] loss: 4.148, train_acc: 0.996, test_acc: 0.994
[epoch 9] loss: 2.102, train_acc: 0.998, test_acc: 0.996
[epoch 10] loss: 1.252, train_acc: 0.999, test_acc: 0.996


In [36]:
testloader = DataLoader(testset, batch_size=256, 
                        collate_fn=create_mini_batch)
predictions = get_predictions(model, testloader)

df = pd.DataFrame({"Category": predictions.tolist()})

df.to_csv('bert_1_prec_training_samples.csv', index=False)
df.head()

,Category
0,0
1,1
2,0
3,0
4,1
